---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

### Import the necessary libraries

In [20]:
#%matplotlib notebook
#import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

### Feature Extraction

Function to fine-tune the features in both training and test datasets

In [21]:
def feature_extraction(blight_data):
    
    # Remove unwanted features from training data
    blight_data.drop(['inspector_name','violation_zip_code','non_us_str_code','grafitti_status','ticket_issued_date',
                 'hearing_date','violation_description','state_fee','admin_fee','violation_code', 'clean_up_cost',
                 'violator_name','violation_street_name','mailing_address_str_name','city','zip_code','violation_street_number',
                'mailing_address_str_number'],axis=1, inplace=True)
    print('Blight Data Shape ', blight_data.shape)
    blight_data.describe(include='all').T
    
    # Convert the 'State' feature into a column with binary value for 'out_of_state'
    # Remove the original 'State' column
    blight_data['out_of_state'] = (blight_data['state']!= 'MI') * 1
    blight_data.drop(['state'], axis=1,inplace=True)
    blight_data['out_of_country'] = (blight_data['country']!= 'USA') * 1
    blight_data.drop(['country'], axis=1,inplace=True)
    print('Blight Data Shape ', blight_data.shape)
    blight_data.describe(include='all').T
    
    # Fill all the NaN values with 'mean' value in the columns
    blight_data.fillna(inplace=True, value={'fine_amount':blight_data['fine_amount'].mean(),'late_fee':blight_data['late_fee'].mean(),
                                      'judgment_amount':blight_data['judgment_amount'].mean()})
    
    # Convert the 'String values into One-Hot vector
    blight_OneHot = pd.get_dummies(blight_data, columns=["agency_name","disposition"])
    print('Blight Data Shape ', blight_OneHot.shape)
    blight_OneHot.describe(include='all').T
    
    return blight_OneHot

### Analyze the Features

Get the details of the features in the training data

Remove the columns that have too many null values, and too many unique values

In [24]:
from sklearn.metrics import roc_auc_score

def blight_model(): 

    # Load the training and test datasets
    train_data = pd.read_csv('train.csv',sep=',', engine='python', index_col='ticket_id')
    print('Training Data Shape ', train_data.shape)
    test_data = pd.read_csv('test.csv', sep=',', engine='python',index_col='ticket_id')
    print('Testing Data Shape ', test_data.shape)

    # Extract training data target column
    train_data = train_data[np.isfinite(train_data['compliance'])]
    train_target = train_data.iloc[:,-1]
    #print(train_target.head())
    #print(train_target.shape)
    
    # Find the columns that are available during the prediction in real world
    train_cols = train_data.columns
    test_cols = test_data.columns
    fin_cols = train_cols.intersection(test_cols)

    # Remove columns that present only in training set and not in test set
    train_data = train_data[fin_cols]
    
    # To avoid haing different number of columns in the training and test sets after get_dummies step, 
    # concatenate taining and test datasets, and apply the feature extraction then split them to the original length
    train_objs_num = len(train_data)
    print(train_objs_num)
    dataset = pd.concat(objs=[train_data, test_data], axis=0)
    dataset_preprocessed = feature_extraction(dataset)
    
    train_preprocessed = dataset_preprocessed[:train_objs_num]
    print("train_preprocessed shape", train_preprocessed.shape)
    test_preprocessed = dataset_preprocessed[train_objs_num:]
    print("test preprocessed shape", test_preprocessed.shape)
    
    #######  BUILDING THE MODEL
    
    #Split the traing data into train and test datasets
    train_X, test_X, train_y, test_y = train_test_split(train_preprocessed, train_target, random_state = 35)
    
    # DecisionTreeClassifier model fitting
    dtc_model = DecisionTreeClassifier()
    dtc_model.fit(train_X, train_y)
    #train_predict = dtc_model.predict(train_X)
    #test_predict = dtc_model.predict(test_X)
    
    # Calculate the prediction probability and AUC score for test dataset provided in the problem
    #test_proba = dtc_model.predict_proba(test_X)
    #print(test_OneHot.shape)
    result_proba = dtc_model.predict_proba(test_preprocessed)
    result = pd.Series(data=result_proba[:,1], index=test_preprocessed.index)
    
    return result

In [25]:
blight_model()

Training Data Shape  (250306, 33)
Testing Data Shape  (61001, 26)
159880
Blight Data Shape  (220881, 8)
Blight Data Shape  (220881, 8)
Blight Data Shape  (220881, 19)
train_preprocessed shape (159880, 19)
test preprocessed shape (61001, 19)


ticket_id
284932    0.049830
285362    0.011338
285361    0.065244
285338    0.049830
285346    0.065244
285345    0.049830
285347    0.055072
285342    0.927778
285530    0.028133
284989    0.045372
285344    0.055072
285343    0.011338
285340    0.011338
285341    0.055072
285349    0.065684
285348    0.049223
284991    0.045372
285532    0.019151
285406    0.019151
285001    0.011703
285006    0.000000
285405    0.028133
285337    0.045372
285496    0.055072
285497    0.049830
285378    0.028133
285589    0.045372
285585    0.049830
285501    0.065244
285581    0.011338
            ...   
376367    0.026846
376366    0.029437
376362    0.185242
376363    0.333333
376365    0.026846
376364    0.029437
376228    0.029437
376265    0.029437
376286    0.252495
376320    0.029437
376314    0.029437
376327    0.252495
376385    0.252495
376435    0.500000
376370    0.932143
376434    0.071726
376459    0.045508
376478    0.000000
376473    0.029437
376484    0.013752
376482    0.016043
37